## Import Packages

In [2]:
import sys
import spotipy
import spotipy.util as util
from spotipy.client import SpotifyException

from collections import Counter
from gensim.utils import tokenize

import time
import json
import ndjson
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import requests

import creds

## Define Get Playlists

In [3]:
def find_playlists(session, w, max_count=1950):
    try:
        res = session.search(w, limit=50, type='playlist')
        while res:
            for playlist in res['playlists']['items']:
                yield playlist
                max_count -= 1
                if max_count == 0:
                    return
            tries = 3
            while tries > 0:
                try:
                    res = session.next(res['playlists'])
                    tries = 0
                except SpotifyException as e:
                    tries -= 1
                    time.sleep(0.2)
                    if tries == 0:
                        raise
    except SpotifyException as e:
        status = e.http_status
        if status == 404:
            raise StopIteration
        raise

## Get Playlists from List of Terms

In [4]:
terms = ["a",
         "discography",
         "collection",
         "soundtrack",
         "rock",
         "hiphop",
         "jazz",
         "metal", 
         "classical",
         "pop",
         "country",
         "blues",
         "rap",
         "hits",
         "top",
         "workout",
         "my",
         "playlist",
         "mood",
         "hype",
         "spotify",
         "vibe",
         "musical",
         "recent",
         "night",
         "day",
         "positive",
         "all",
         "dance",
         "latin",
         "southern",
         "electropop",
         "tropical",
         "contemporary",
         "alternative",
         "gangster",
         "soul",
         "house",
         "progressive",
         "modern"
        ]

In [5]:
playlists = {}
words_seen = set()
count = 0
dupes = 0

In [6]:
# Create a Spotify instance. This will refresh the authorization as it will expire every hour.
session = creds.session

In [7]:
for term in terms:
    # Set the counter for the search term
    count_term = 0
    
    # Set the Counter for popular terms and only proceed with unique search terms
    if not term in words_seen:
        word_counts = Counter({term : 1})
    
        # Get 50,000 playlists for each unique term
        while count_term < 50000:
            for word, _ in word_counts.most_common():
                if not word in words_seen:
                    words_seen.add(word)
                    print(f"Total playlist count is {count}...")
                    print(f"Term playlist count is {count_term}...")
                    print('word>', word)
                    print("")
                    for playlist in find_playlists(session, word):
                        if playlist['id'] in playlists: 
                            dupes += 1
                        elif playlist['name'] and playlist['owner']: 
                            playlists[playlist['id']] = {
                                  'owner': playlist['owner']['id'],
                                  'name': playlist['name'],
                                  'id': playlist['id'],
                            }
                            count += 1
                            count_term += 1
                            for token in tokenize(playlist['name'], lowercase=True): 
                                word_counts[token] += 1
                    break
    else:
        print("Repeated Term")
        print("")

Total playlist count is 0...
Term playlist count is 0...
word> a



KeyboardInterrupt: 

In [4]:
# Save dictionary of playlists as a JSON file.
with open("./data/playlists.json", "w") as write_file:
    json.dump(playlists, write_file)

In [2]:
# Open playlists JSON and save as dictionary
with open("./data/playlists.json", "r") as playlists_file:
    playlists = json.load(playlists_file)